---
# Caso Limpieza de Sueldos de San Francisco

Después del análisis de datos que usted realizó sobre la nómina de sueldos de San Francisco, el alcalde notó los errores de datos que existían y los problemas que acarrean a la gestión. Por este motivo, decidió contratarlo para que haga un análisis de la calidad de datos y realice la limpieza de datos a efecto de contar con información confiable para la toma de decisiones. 


Como entregable, se solicita un notebook ordenado, documentado y reproducible en donde:

Realice un análisis de valores nulos en las columnas BasePay, OverTimePay, OtherPay, Benefits, TotalPay y TotalPayBenefits
Realice un análisis de outliers en la columna BasePay
Realice la limpieza de los datos de acuerdo a los criterios aprendidos (dejar, imputar, eliminar)
Genere un nuevo archivo Excel con los datos limpios


---

In [7]:
import pandas as pd

df = pd.read_csv("Salaries (1).csv")
df

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148649,148650,Roy I Tillery,Custodian,0.00,0.00,0.00,0.0,0.00,0.00,2014,NaN,San Francisco,NaN
148650,148651,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,NaN,San Francisco,NaN
148651,148652,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,NaN,San Francisco,NaN
148652,148653,Not provided,Not provided,NaN,NaN,NaN,NaN,0.00,0.00,2014,NaN,San Francisco,NaN


In [23]:
#1) Realice un análisis de valores nulos en las columnas BasePay, OvertimePay, OtherPay, Benefits, TotalPay y TotalPayBenefits
valores = ['BasePay','OvertimePay','OtherPay','Benefits','TotalPay','TotalPayBenefits']
cantidad_nulo = []

for i in valores: 
    cantidad_nulo = df[i].isnull().sum()
    print(f"Valores nulos en {i} : {cantidad_nulo}")

Valores nulos en BasePay : 609
Valores nulos en OvertimePay : 4
Valores nulos en OtherPay : 4
Valores nulos en Benefits : 36163
Valores nulos en TotalPay : 0
Valores nulos en TotalPayBenefits : 0


In [21]:
#2) Realice un análisis de outliers en la columna BasePay

#Primero eliminaremos los valores nulos
basepay_filtrado = df['BasePay'].dropna()

#Calcularemos los limites del quintil para saber desde que punto se consideran outliers
Q1 = basepay_filtrado.quantile(0.25)
Q3 = basepay_filtrado.quantile(0.75)
IQR = Q3 - Q1
LSUP = Q1 - 1.5 * IQR
LINF = Q3 + 1.5 * IQR

#Calculamos los outliers
outliers = basepay_filtrado[(basepay_filtrado > LSUP) | (basepay_filtrado < LINF)]
print("Cantidad de outliers:", len(outliers))
print(outliers)

Cantidad de outliers: 148045
0         167411.18
1         155966.02
2         212739.13
3          77916.00
4         134401.60
            ...    
148645         0.00
148647         0.00
148648         0.00
148649         0.00
148653         0.00
Name: BasePay, Length: 148045, dtype: float64


In [22]:
#3) Realice la limpieza de los datos de acuerdo a los criterios aprendidos (dejar, imputar, eliminar)
# ['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay','Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency','Status'],

valores = ['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay','Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency','Status']
cantidad_nulo = []

def calcular_vacios(dataframe):
    for i in valores:
        if i in dataframe.columns: 
            cantidad_nulo = dataframe[i].isnull().sum()
            print(f"Valores nulos en {i} : {cantidad_nulo}")

#calcular_vacios(df)


#Eliminare las columnas Notes y Status porque estan vacias y no contienen datos relevantes
df_limpia = df.drop(columns=['Notes','Status','BasePay'])

#Eliminare los 4 valores vacios de OvertimePay y OtherPay porque revisandolo no tienen datos
df_limpia = df_limpia.dropna(subset=['OvertimePay'])

#Calcularemos el valor de benefits en los que esta vacio porque tenemos los datos para hacerlo
df_limpia['Benefits'] = df_limpia['Benefits'].fillna(df_limpia['TotalPayBenefits'] - df_limpia['TotalPay'])


#Cambiare los valores nulos de Overtime Pay y OtherPay a 0 porque asumiremos que no tienen
df_limpia['OtherPay'] = df_limpia['Benefits'].fillna(0)
df_limpia['OvertimePay'] = df_limpia['Benefits'].fillna(0)


#calcular_vacios(df_limpia)
print(df_limpia)



            Id       EmployeeName  \
0            1     NATHANIEL FORD   
1            2       GARY JIMENEZ   
2            3     ALBERT PARDINI   
3            4  CHRISTOPHER CHONG   
4            5    PATRICK GARDNER   
...        ...                ...   
148645  148646   Carolyn A Wilson   
148647  148648     Joann Anderson   
148648  148649        Leon Walker   
148649  148650      Roy I Tillery   
148653  148654          Joe Lopez   

                                              JobTitle  OvertimePay  OtherPay  \
0       GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY          0.0       0.0   
1                      CAPTAIN III (POLICE DEPARTMENT)          0.0       0.0   
2                      CAPTAIN III (POLICE DEPARTMENT)          0.0       0.0   
3                 WIRE ROPE CABLE MAINTENANCE MECHANIC          0.0       0.0   
4         DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)          0.0       0.0   
...                                                ...          ...  

In [ ]:
#4) Genere un nuevo archivo Excel con los datos limpios